## MPP Problem for Practical Project

# STEADY FREE CONVECTION IN A RECTANGULAR CAVITY FILLED WITH A POROUS MEDIUM

### Exploratory notebook

This notebook is used to offer a reference sequential Jacobi implementation of the simulator and to visualize to results obtained.

### Simulation parameters

In a left-handed Cartesian coordinate system, we assume a cavity of height ``h`` along the z axis, divided in ``N`` cells (``N3``), and lengths ``Ax*h``, ``Ay*h``, where ``Ax`` and ``Ay`` are aspect ratios for the cavity's dimensions. For simplification, in continuous form, ``0<=h<=1``. The cells are considered form an equidistant grid, so that ``dx=dy=dz``.

Two walls (``x==0`` and ``x==Ax*h``) are considered to have a constant ``T0`` temperature, though this can be parametrized to support a "hot wall" (``Th`` at ``x==0``) and a "cold wall" (``Tc`` at ``x==Ax*h``). 

Kinematic viscosity along z axis is not accounted for and considered to be 0.

In [1]:
# libraries
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy

# simulation parameters
Ra = 1000
T0 = 0
Th = 0 # can swap for custom temperature, e.g. 1
Tc = 0 # can swap for custom temperature, e.g. 0

# cavity parameters
h = 1
N = 25
Ax = 1
Ay = 1

N1 = N * Ax
N2 = N * Ay
N3 = N

# cell size
d = h / N

# grid
u = np.zeros((N1, N2, N3))
v = np.zeros((N1, N2, N3))
t = np.zeros((N1, N2, N3))

# constant temperature walls condition
t[0,:,:] = np.full((N2, N3), Th)
t[N1-1,:,:] = np.full((N2, N3), Tc)

# auxiliary grid
u_new = deepcopy(u)
v_new = deepcopy(v)
t_new = deepcopy(t)

### Simulation
Results of the simulation will be placed in a folder ``output/`` (please make sure it exists before running the cell).

Jacobi iteration

In [2]:
err_u=0
err_v=0
err_t=0

def jacobi_iteration():
    global u, v, t, u_new, v_new, t_new, err_u, err_v, err_t

    for i in range(1, N1 - 1):
        for j in range(1, N2 - 1):
            for k in range(1, N3 - 1):
                u_new[i,j,k] = Ra*d/12 * (t[i,j+1,k] - t[i,j-1,k]) + \
                    1/6 * (
                        u[i-1,j,k] + u[i+1,j,k] + \
                        u[i,j-1,k] + u[i,j+1,k] + \
                        u[i,j,k-1] + u[i,j,k+1]
                    )
                
                v_new[i,j,k] = Ra*d/12 * (t[i+1,j,k] - t[i-1,j,k]) + \
                    1/6 * (
                        v[i-1,j,k] + v[i+1,j,k] + \
                        v[i,j-1,k] + v[i,j+1,k] + \
                        v[i,j,k-1] + v[i,j,k+1]
                    )
                
                t_new[i,j,k] = 1/6 * ( \
                    t[i-1,j,k] + t[i+1,j,k] + \
                    t[i,j-1,k] + t[i,j+1,k] + \
                    t[i,j,k-1] + t[i,j,k+1] + \
                d * d \
                - \
                1/4 * (
                    (u[i,j,k+1]-u[i,j,k-1]) * (t[i,j+1,k]-t[i,j-1,k]) - \
                    (u[i,j+1,k]-u[i,j-1,k]) * (t[i,j,k+1]-t[i,j,k-1]) \
                    + \
                    (v[i+1,j,k]-v[i-1,j,k]) * (t[i,j,k+1]-t[i,j,k-1]) - \
                    (v[i,j,k+1]-v[i,j,k-1]) * (t[i+1,j,k]-t[i-1,j,k])
                    )
                )
                # error evaluation
                erru = abs(u[i,j,k] - u_new[i,j,k])
                errv = abs(v[i,j,k] - v_new[i,j,k])
                errt = abs(t[i,j,k] - t_new[i,j,k])
                err_u = max(err_u, erru)
                err_v = max(err_v, errv)
                err_t = max(err_t, errt)

    # adiabatic wall conditions
    #  bottom/top walls => constant z
    t_new[:,:,0] = deepcopy(t[:,:,1])
    t_new[:,:,N3-1] = deepcopy(t[:,:,N3-2])
    #  back/front walls => constant y (LHS)
    #  or left/right if RHS
    t_new[:,0,:] = deepcopy(t[:,1,:])
    t_new[:,N2-1,:] = deepcopy(t[:,N2-2,:])
    
    # swap
    (u, u_new) = (u_new, u)
    (v, v_new) = (v_new, v)
    (t, t_new) = (t_new, t)

Plotting functions

In [3]:
X, Y, Z = np.meshgrid(np.arange(N1), np.arange(N2), -np.arange(N3))

def plotContourCube():
    fig = plt.figure()
    ax = fig.add_subplot(111, projection="3d")

    intv = 10
    vmin = 0 #t.min()
    vmax = 1 #t.max()

    kw = {
        'vmin': vmin,
        'vmax': vmax,
        'levels': np.linspace(vmin, vmax, 100),
    }

    _ = ax.contourf(
        X[:, :, 0], Y[:, :, 0], t[:, :, 0],
        zdir='z', offset=0, **kw
    )
    _ = ax.contourf(
        X[0, :, :], t[0, :, :], Z[0, :, :],
        zdir='y', offset=0, **kw
    )
    C = ax.contourf(
        t[:, -1, :], Y[:, -1, :], Z[:, -1, :],
        zdir='x', offset=X.max(), **kw
    )

    xmin, xmax = X.min(), X.max()
    ymin, ymax = Y.min(), Y.max()
    zmin, zmax = Z.min(), Z.max()
    ax.set(xlim=[xmin, xmax], ylim=[ymin, ymax], zlim=[zmin, zmax])

    edges_kw = dict(color='0.4', linewidth=1, zorder=1e3)
    ax.plot([xmax, xmax], [ymin, ymax], 0, **edges_kw)
    ax.plot([xmin, xmax], [ymin, ymin], 0, **edges_kw)
    ax.plot([xmax, xmax], [ymin, ymin], [zmin, zmax], **edges_kw)
    
    # account for LHS-RHS difference
    ax.set(
        xlabel='Y',
        ylabel='X',
        zlabel='Z'
    )

    ax.view_init(40, -30, 0)
    ax.set_box_aspect(None, zoom=0.9)

    fig.colorbar(C, ax=ax, fraction=0.02, pad=0.1, label='Temperature')

Manual iteration

In [4]:
#jacobi_iteration()

In [5]:
#plotContourCube()

In [6]:
#plt.close()

Main loop

In [7]:
# termination conditions
#  maximum amount of iterations if not yet converged
max_iter = 1000
#  epsilon
eps = 1e-9

iter = 0
stop = False

while not stop:
    # plot
    if iter % 10 == 0:
        plotContourCube()
        plt.savefig(f"output/t{iter}")
        plt.close()

    jacobi_iteration()
    
    iter += 1
    if (iter >= max_iter):
        stop = True
    elif (err_u < eps and err_v < eps and err_t < eps):
        stop = True